In [74]:
import cv2
import numpy as np
import pandas as pd

### 1. Data Processing

In [75]:
train_df = pd.read_csv("mnist_train_final.csv")
test_df = pd.read_csv("mnist_test_final.csv")

In [76]:
train_df.shape

(59999, 785)

### 2. Create Model

In [77]:
model = cv2.ml.ANN_MLP_create()
model.setLayerSizes(np.array([784, 64, 10]))
model.setActivationFunction(cv2.ml.ANN_MLP_SIGMOID_SYM, 0.6, 1.0)
model.setTrainMethod(cv2.ml.ANN_MLP_BACKPROP, 0.1, 0.1)
model.setTermCriteria((cv2.TERM_CRITERIA_MAX_ITER | cv2.TERM_CRITERIA_EPS, 100, 1.0))

### 3. Train Model

In [78]:
epochs = 10

for epoch in range(epochs):

    counter = 0

    for img in train_df.values: # per img in train dataset
        if counter > train_df.shape[0]:
            break

        counter += 1

        ys = np.zeros(10, dtype=np.float32)
        ys[img[1:][0]] = 1

        data = cv2.ml.TrainData_create(
            np.array([img[1:]], dtype=np.float32), # x
            cv2.ml.ROW_SAMPLE,
            np.array([ys], dtype=np.float32) # y
        )

        # fit model
        if model.isTrained():
            model.train(data, cv2.ml.ANN_MLP_UPDATE_WEIGHTS | cv2.ml.ANN_MLP_NO_INPUT_SCALE | cv2.ml.ANN_MLP_NO_OUTPUT_SCALE)
        else:
            model.train(data, cv2.ml.ANN_MLP_NO_INPUT_SCALE | cv2.ml.ANN_MLP_NO_OUTPUT_SCALE)
    
    print(f"Epoch {epoch}")

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9


### 4. Evaluate Model

In [79]:
def sigmoid(x):
    return 1 / (1+np.e**(-x))

In [80]:
def predict(model, img):
    if img.shape != (784, ):
        if img.shape != (28, 28):
            img = cv2.resize(img, (28, 28), interpolation=cv2.INTER_LINEAR)
        img = img.reshape(784, )
    
    results = model.predict(np.array([img], dtype=np.float32))[1]

    for i in range(len(results)):
        results[i] = np.round(sigmoid(results[i]))
    

    return results

In [86]:
predict(model, train_df.drop(["label"], axis=1).iloc[1].values)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

terrible model wtf bro